# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [17]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [18]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [19]:
# confirm that our new database was created

database_names = mongo.list_database_names()
if "uk_food" in database_names:
    print("Database exists!")
else:
    print("Database does not exist.")

Database exists!


In [20]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [21]:
# review the collections in our new database
establishments = db.list_collection_names()

# print collection names
print("Collections in", db, "database:")
for name in establishments:
    print("-", name)

Collections in Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food') database:
- establishments


In [22]:
# review a document in the establishments collection
collection = db["establishments"]
document = collection.find_one()

# print the document
print("Document in 'establishments' collection:")
print(document)

Document in 'establishments' collection:
{'_id': ObjectId('660488a91bc98f41d15b6487'), 'name': 'Penang Flavours', 'address': '123 Greenwich Street', 'city': 'Greenwich', 'postcode': 'SE10 8JA', 'cuisine': 'Halal', 'rating': None}


In [23]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [24]:
# Create a dictionary for the new restaurant data

new_restaurant_data = {
    "name": "Penang Flavours",
    "address": "123 Greenwich Street",
    "city": "Greenwich",
    "postcode": "SE10 8JA",
    "cuisine": "Halal",
    "rating": None  
}


In [25]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant_data)


InsertOneResult(ObjectId('660488f11bc98f41d15b6489'), acknowledged=True)

In [26]:
# Check that the new restaurant was inserted
if insert_result.acknowledged:
    print("Penang Flavours has been successfully added to the database.")
else:
    print("Failed to add Penang Flavours to the database.")

NameError: name 'insert_result' is not defined

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [27]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
projections = {'BusinessTypeID': 1, 'BusinessType': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [28]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessTypeID': 1},
    {'$set': {'BusinessType': 1}} 
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [29]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"name":"Penang Flavours"}))


{'_id': ObjectId('660488a91bc98f41d15b6487'),
 'address': '123 Greenwich Street',
 'city': 'Greenwich',
 'cuisine': 'Halal',
 'name': 'Penang Flavours',
 'postcode': 'SE10 8JA',
 'rating': None}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [30]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
pprint({"Number of documents with LocalAuthorityName as 'Dover'": count})


{"Number of documents with LocalAuthorityName as 'Dover'": 0}


In [31]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete documents with LocalAuthorityName as "Dover"
delete_result = collection.delete_many(query)
pprint({"Number of documents deleted where LocalAuthorityName is 'Dover'": delete_result.deleted_count})

{"Number of documents deleted where LocalAuthorityName is 'Dover'": 0}


In [32]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Count documents with LocalAuthorityName as "Dover"
count = collection.count_documents(query)
pprint({"Number of documents with LocalAuthorityName as 'Dover'": count})

{"Number of documents with LocalAuthorityName as 'Dover'": 0}


In [34]:
# Check that other documents remain with 'find_one'
remaining_document = collection.find_one()

# If no other documents
if remaining_document:
    print("At least one document remains in the collection.")
    print("Here is a sample document:")
    print(remaining_document)
else:
    print("No documents remain in the collection.")


At least one document remains in the collection.
Here is a sample document:
{'_id': ObjectId('660488a91bc98f41d15b6487'), 'name': 'Penang Flavours', 'address': '123 Greenwich Street', 'city': 'Greenwich', 'postcode': 'SE10 8JA', 'cuisine': 'Halal', 'rating': None}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [35]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"},
                                           "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}
                                         }} ])

UpdateResult({'n': 2, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [36]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [37]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

UpdateResult({'n': 2, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [38]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()
latitude_type = type(sample_document.get("Latitude"))
longitude_type = type(sample_document.get("Longitude"))
rating_value_type = type(sample_document.get("RatingValue"))
pprint(establishments.find_one())

{'RatingValue': None,
 '_id': ObjectId('660488a91bc98f41d15b6487'),
 'address': '123 Greenwich Street',
 'city': 'Greenwich',
 'cuisine': 'Halal',
 'geocode': {'latitude': None, 'longitude': None},
 'name': 'Penang Flavours',
 'postcode': 'SE10 8JA',
 'rating': None}
